<a href="https://colab.research.google.com/github/kleinfossil/googlecolabfree_stable_diffusion/blob/main/Adjusted_alpaca_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Inital Setup
#@markdown This installs requirements and downloads the models
Basemodel = "decapoda-research/llama-7b-hf" #@param ["decapoda-research/llama-7b-hf"]
LoRa = "tloen/alpaca-lora-7b" #@param ["tloen/alpaca-lora-7b"]
from IPython.display import clear_output

!pip install -q  git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip -q install langchain
clear_output()

from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from peft import PeftModel
import torch

tokenizer = LlamaTokenizer.from_pretrained(Basemodel)
lora_weights = LoRa

base_model = LlamaForCausalLM.from_pretrained(
    Basemodel,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map='auto',
)
base_model = PeftModel.from_pretrained(
    base_model,
    lora_weights,
    torch_dtype=torch.float16,
    device_map="auto"
    )
clear_output()

In [ ]:
#@markdown # Setup Pipeline
#@markdown This is somehow not working correctly. Need to check how to change it. 
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)
clear_output()

In [ ]:
#@markdown # Chat without Memory
#@markdown This will not use a memory
from langchain import PromptTemplate, LLMChain

template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])
clear_output()
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

In [ ]:
singel_prompt = "Tell me a joke, but not a knock-knock joke!" #@param {type:"string"}

print(llm_chain.run(singel_prompt))

In [ ]:
#@markdown # Chat with memory
#@markdown This will setup a chat with memory
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# We are going to set the memory to go back 4 turns
memory = 4 #@param {type:"integer"}
window_memory = ConversationBufferWindowMemory(k=memory)

conversation = ConversationChain(
    llm=local_llm, 
    memory=window_memory,
    verbose=True
)
conversation.prompt.template
clear_output()

In [ ]:
Prompt = "Can you make the story funny? " #@param {type:"string"}
conversation.predict(input=Prompt)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me a happy story with 50 words. 
AI:  Once upon a time there was a little girl who loved playing in her garden. She would spend hours digging up worms and planting seeds. One day she found a special seed that grew into a beautiful flower. Everyone admired how pretty it looked so they decided to make it their own. They all took turns caring for this magical flower until one day it bloomed! It filled everyone's hearts with joy as it spread happiness throughout the town.
Human: Can you make the story funny? 
AI:

> Finished chain.


' Once upon a time there was a little boy named Bob who really wanted to be a pirate but he had no idea what being a real life pirate entailed. So he went on a quest to find out more about them. He read books, watched movies, and even tried to dress like one. But nothing seemed right. Until finally his mom said "Why don’t we just go visit some actual pirates?" And off they set sail to meet these mysterious people. When they arrived at their destination, Bob was surprised by how much smaller than expected they were. In fact, they weren\'t very impressive looking at all. Still determined, Bob asked if he could join them on their next voyage. To which they replied “Sure!” Little did he know that this journey would change him forever...'